In [1]:
import mediapipe as mp
import cv2
import os
import numpy as np

In [2]:
SAVE_PATH = r"C:/Users/ronde/PROJECTS/ASL_TO_TEXT_FILES/data/gesture_sequences"
IMG_SIZE = 320
SEQUENCE_LENGTH = 30  # Number of frames to capture for each gesture

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)

def normalize_landmarks(landmarks):
    x_coords = [lm[0] for lm in landmarks]
    y_coords = [lm[1] for lm in landmarks]
    min_x, max_x = min(x_coords), max(x_coords)
    min_y, max_y = min(y_coords), max(y_coords)
    normalized_landmarks = [
        [(x - min_x) / (max_x - min_x), (y - min_y) / (max_y - min_y), z]
        for x, y, z in landmarks
    ]
    return normalized_landmarks

def preprocess_landmarks(landmarks, img_size=IMG_SIZE):
    normalized_landmarks = normalize_landmarks(landmarks)
    landmarks_image = np.zeros((img_size, img_size, 3), dtype=np.uint8)
    for lm in normalized_landmarks:
        x, y = int(lm[0] * (img_size - 1)), int(lm[1] * (img_size - 1))
        cv2.circle(landmarks_image, (x, y), 10, (255, 0, 0), 3)
    return landmarks_image

gesture_name = input("Enter the gesture name to train: ").strip()
output_folder = os.path.join(SAVE_PATH, gesture_name)
os.makedirs(output_folder, exist_ok=True)

cam = cv2.VideoCapture(0)
if not cam.isOpened():
    print("Error: Camera not accessible.")
    exit()

sequence_count = 0
frame_count = 0
capturing = False
current_sequence = []

while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame.")
        break
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = [(landmark.x, landmark.y, landmark.z) for landmark in hand_landmarks.landmark]
            preprocessed_image = preprocess_landmarks(landmarks)

            cv2.imshow('Preprocessed Hand', preprocessed_image)

            if capturing:
                current_sequence.append(preprocessed_image)
                frame_count += 1

                if frame_count == SEQUENCE_LENGTH:
                    sequence_name = os.path.join(output_folder, f"{gesture_name}_{sequence_count}")
                    os.makedirs(sequence_name, exist_ok=True)
                    for i, img in enumerate(current_sequence):
                        cv2.imwrite(os.path.join(sequence_name, f"frame_{i}.jpg"), img)
                    print(f"Sequence {sequence_count} saved to: {sequence_name}")
                    sequence_count += 1
                    frame_count = 0
                    current_sequence = []
                    capturing = False

    cv2.imshow('Hand Tracking', frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('c'): 
        capturing = True
        print("Started capturing sequence...")
    elif key == ord('s'): 
        capturing = False
        current_sequence = []
        frame_count = 0
        print("Stopped capturing.")
    elif key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

c:\Users\ronde\PROJECTS\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
